In [1]:
from endpoints import SofaScore
from datetime import datetime, timedelta
from helpers import send_request, generate_date_list, generate_live_games, fetch_event_data
import pandas as pd
import matplotlib.pyplot as plt


# Establish todays date
start_date = datetime.today().strftime('%Y-%m-%d')

# start_date = '2023-01-20'

# Establish an end date
end_date = '2023-01-01'

# Generate a list of dates
dates = generate_date_list(start_date,end_date)

# Create a list of endpoints from the list of dates and the Sofa base_endpoint
dated_endpoints = [SofaScore.BASE_ENDPOINT + date for date in dates]



In [2]:
from helpers import send_request, generate_date_list, generate_live_games, fetch_event_data

# store list of responses from the urls
responses = [response for response in await fetch_event_data(dated_endpoints) if response['events']]


In [3]:
def extract_event_data(response_array):

    # Initialize an empty list to store the event data
    event_data = []

    for response in response_array:
        # Filter out events that are not postponed

        events = [event for event in response['events'] if event['status']['type'] not in ['postponed', 'canceled']]
        
        # Loop through each event
        for event in events:   
                    
            # sometime score dicts have multiple scores, so just take max value; they should be same. 
            score_key = max(event['homeScore'], key = event['homeScore'].get) if event['homeScore'] else None
            
            # if no score obj, then skip
            if score_key:
                # Create a dictionary to store the relevant data for each event
                struct = {
                    'tournament' : event['tournament']['name'],  # Name of the tournament
                    'region' : event['tournament']['category']['name'],  # Region of the tournament
                    'home_team' : event['homeTeam']['name'],  # Name of the home team
                    'away_team' : event['awayTeam']['name'],  # Name of the away team
                    'home_score': event['homeScore'][score_key],  # Current score of the home team
                    'away_score': event['awayScore'][score_key],  # Current score of the away team
                    'total'     : event['homeScore'][score_key] + event['awayScore'][score_key],  # Total score
                    'start_timeestamp' : event['startTimestamp'],  # Start timestamp of the event
                    'game_id' : event['id'] # Unique Game ID
                    # 'start_datetime' : convert_timestamp_to_est_date(event['startTimestamp'])  # Start datetime of the event in EST
                }
                
                # Append the dictionary to the list of event data
                event_data.append(struct)
    
    # Return the list of event data
    return event_data

In [4]:
event_data = extract_event_data(responses)

In [5]:
# create endpoints to each specific games statistics data

from endpoints import SofaScore

def extract_specific_stats(events):
    unique_id_urls = []
    for games in events:
        game_id_url = SofaScore.STAT_ENDPOINT.replace('UNIQUEGAMEID', str(games['game_id']))
        unique_id_urls.append(game_id_url)
    


    return unique_id_urls

stat_id_urls = extract_specific_stats(event_data)

In [6]:
# Function to gather each games individual response

from endpoints import SofaScore

def extract_specific_games(events):
    game_urls = []

    for games in events:
        game_url = SofaScore.INDIVIDUAL_ENDPOINT + str(games['game_id'])
        game_urls.append(game_url)
    
    return game_urls

individual_game_urls = extract_specific_games(event_data)

In [7]:
print(len(individual_game_urls))

108079


In [ ]:

# import asyncio


# async def fetch_stat_game_responses(stat_id_urls):
 
    
#     stat_game_responses = []
    
#     for i in range(5):
#         start = 0
#         count = 500
#         indRep = await fetch_event_data(stat_id_urls[start:count])
#         for item in indRep:
#             stat_game_responses.append(item)
#         count += 500
#         start += 500
#         await asyncio.sleep(10)
    
#     return stat_game_responses

# stat_response_results = await fetch_stat_game_responses(stat_id_urls)

In [8]:
# Fetch indivudal game data if it has the 'hasXg' present in the response

import asyncio

individual_responses = []
for start in range(0, 10000, 500):
    count = start + 500
    ind_Rep = await fetch_event_data(individual_game_urls[start:count])
    individual_responses.extend(ind_Rep)
    await asyncio.sleep(10)




In [9]:
print(len(individual_responses))

10000


In [ ]:
# Iterate over individual game data to gather dictionary for SofaScores expectations on the game

def expectations_from_stats(stat_responses):
    expectations_response = []
    
    for response in stat_responses:
        if 'statistics' in response:
            for stats in response['statistics']:
                for groups in stats['groups']:
                    for expectations in groups['statisticsItems']:
                        if expectations['name'] == 'Expected goals':
                            struct = {
                                'type_of_stat' : expectations['name'],  # Name of the tournament
                                'home_team_expectation' : expectations['home'],  # Name of the home team
                                'away_team_expectation' : expectations['away'],  # Name of the away team
                            }
                            
                            expectations_response.append(struct)
                        else:
                            continue
    
    return expectations_response


                

In [11]:

# Function to gather game data + data from the statistics API. Different from extract_event_data

import time


def extract_data_with_xg(response_array):

    # Initialize an empty list to store the event data
    xg_data = []


    # Iterate over each response and item together.
    for response in response_array:

        struct = {
            'tournament' : response['event']['tournament']['name'],  # Name of the tournament
            'region' : response['event']['tournament']['category']['name'],  # Region of the tournament
            'home_team' : response['event']['homeTeam']['name'],  # Name of the home team
            'away_team' : response['event']['awayTeam']['name'],  # Name of the away team
            'home_score': response['event']['homeScore']['current'],  # Current score of the home team
            'away_score': response['event']['awayScore']['current'],  # Current score of the away team
            'total'     : response['event']['homeScore']['current'] + response['event']['awayScore']['current'],  # Total score
            'game_id' : response['event']['id'],
        }

        xg_data.append(struct)

    return xg_data

parsed_response = extract_data_with_xg(individual_responses)

# print(parsed_response)

In [12]:
for x in parsed_response[:5]:
    print(x)

{'tournament': 'Premier League', 'region': 'England', 'home_team': 'Brighton & Hove Albion', 'away_team': 'Tottenham Hotspur', 'home_score': 4, 'away_score': 2, 'total': 6, 'game_id': 11777280}
{'tournament': 'Premier League', 'region': 'England', 'home_team': 'Arsenal', 'away_team': 'West Ham United', 'home_score': 0, 'away_score': 2, 'total': 2, 'game_id': 11777281}
{'tournament': 'Serie A', 'region': 'Italy', 'home_team': 'Fiorentina', 'away_team': 'Torino', 'home_score': 1, 'away_score': 0, 'total': 1, 'game_id': 11407507}
{'tournament': 'Serie A', 'region': 'Italy', 'home_team': 'Napoli', 'away_team': 'Monza', 'home_score': 0, 'away_score': 0, 'total': 0, 'game_id': 11407513}
{'tournament': 'Serie A', 'region': 'Italy', 'home_team': 'Genoa', 'away_team': 'Inter', 'home_score': 1, 'away_score': 1, 'total': 2, 'game_id': 11407508}


In [ ]:
def calculate_expectation_results(game_data, **kwargs):
    """
    This function compares the games ending home, and away team scores
    to SofaScores algorithmic prediction for both teams.

    Parameters:
    game_data (list): A list of dictionaries containing data about each event.
    **kwargs: Additonal filters to apply on the data. 
    For Example: Testing to see how accurate their predictions are for specific regions
    or teams etc.

    Returns:
    Float: The calculated winrate
        or
    Dictionary: If kwargs given, will return the calculated winrate and kwarg supplied
    within in a dictionary.

    """

    # Convert the event data into a pandas DataFrame
    df = pd.DataFrame(game_data).drop_duplicates().reset_index(drop=True)

    # If there are additional filters, create a query string.
    query = ' & '.join([f"{k} == '{v}'" for k, v in kwargs.items()]) if kwargs else None

    accuracy_test = df.query(f"home_score == round(home_team_expectation) & away_score == round(away_team_expectation)")
    opposite_accuracy_test = df.query(f"home_score != round(home_team_expectation) | away_score != round(away_team_expectation)")

    accuracy_result = df.query(accuracy_test)
    opposite_result = df.query(opposite_accuracy_test)

    winrate = round(accuracy_result.shape[0] / accuracy_result.shape[0] + opposite_result.shape[0], 3)

    struct = {
        'total_games' : accuracy_result.shape[0] + opposite_result.shape[0],
        'accurate_prediction' : accuracy_result.shape[0],
        'accuracy_ratio' : winrate
    }

    return struct


data_results =  calculate_expectation_results(parsed_response)

print(data_results)


In [ ]:
def calculate_winrate(event_data, threshold, **kwargs):
    """
    Credit to: https://github.com/rushcapital
    This function calculates the winrate of the events based on a given threshold.
    Parameters:
    event_data (list): A list of dictionaries containing event data.
    threshold (float): The threshold value to calculate the winrate.
    **kwargs: Additional filters to apply on the data.

    Returns:
    float: The calculated winrate.
    """

    # Convert the event data into a pandas DataFrame
    df = pd.DataFrame(event_data).drop_duplicates().reset_index(drop=True)
    
    # If there are additional filters, create a query string
    query = ' & '.join([f"{k} == '{v}'" for k, v in kwargs.items()]) if kwargs else None
    
    # Create the under and over queries based on the threshold and additional filters
    # The under query selects events where the total score is less than the threshold
    under_query = f"{query} & total < {threshold}" if query else f'total < {threshold}'
    
    # The over query selects events where the total score is more than the threshold
    over_query = f"{query} & total > {threshold}" if query else f'total > {threshold}'
    
    # Apply the under and over queries to the DataFrame
    under = df.query(under_query)
    over = df.query(over_query)
    
    # Calculate the winrate as the ratio of under events to the total number of events
    # If there are no events, the winrate is 0
    winrate = round(under.shape[0] / (under.shape[0] + over.shape[0]),3) if (under.shape[0] + over.shape[0]) != 0 else 0
    
    struct = {
        'total_games' : under.shape[0] + over.shape[0],
        'under_wins' : under.shape[0],
        'under_winrate' : winrate
    }

    return struct


In [ ]:
winrate_england_struct = calculate_winrate(event_data, threshold=3.5, region='England')


print(winrate_england_struct)


In [ ]:
# winrate_struct = calculate_winrate(event_data, threshold=3.5)
import pandas as pd

winrate_england_struct = calculate_winrate(event_data, threshold=3.5, region='England')
winrate_spain_struct = calculate_winrate(event_data, threshold=3.5, region='Spain')
winrate_italy_struct = calculate_winrate(event_data, threshold=3.5, region='Italy')
winrate_belgium_struct = calculate_winrate(event_data, threshold=3.5, region='Belgium')
winrate_greece_struct = calculate_winrate(event_data, threshold=3.5, region='Greece')

Data = pd.DataFrame([winrate_england_struct, winrate_spain_struct, winrate_italy_struct, winrate_belgium_struct, winrate_greece_struct])

import matplotlib.pyplot as plt

regions = ['England', 'Spain', 'Italy', 'Belgium', 'Greece']
winrates = Data['under_winrate']
x = range(len(regions))
plt.bar(regions, winrates)
plt.title('Under 3.5 Winrates by Region')
plt.xlabel('Region')
plt.ylabel('winrate %')
plt.show()




In [ ]:


# function to extract all regions

def extract_regions(response_array):

    unique_regions = []

    # Iterate over the response_array. (This should be event_data)

    for data in response_array:
        region = data['region']

        # If the Region exists in the unique_regions list, skip over it, else, append it.
        if region in unique_regions:
            continue
        else:
            unique_regions.append(region)

    # Return the list.
    return unique_regions

# Call to store the unique list of regions
list_of_regions = extract_regions(event_data)


